In [3]:
!pip install ptflops

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import models, transforms
from PIL import Image
import pandas as pd
import numpy as np
import time
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, roc_auc_score, precision_recall_fscore_support, accuracy_score, roc_curve
from ptflops import get_model_complexity_info

# 1. SETTINGS & PATHS

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMG_DIR = "/kaggle/input/datasets/deadcardassian/pm25vision/train/images/"
CSV_FILE = "cleaned_labels.csv" # Ensure this has no Task 1 duplicates
INPUT_SIZE = (224, 224)
EPOCHS = 50
BACKBONES = ["mobilenet_v2", "resnet50", "inception_v3", "efficientnet_b0", "resnet101"]
SPLITS = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# 2. DATASET & STRATEGY

In [7]:
class LetterboxResize(object):
    def __init__(self, output_size=INPUT_SIZE):
        self.output_size = output_size
    def __call__(self, img):
        canvas = Image.new("RGB", self.output_size, (0, 0, 0))
        img.thumbnail(self.output_size, Image.LANCZOS)
        offset = ((self.output_size[0] - img.size[0]) // 2, (self.output_size[1] - img.size[1]) // 2)
        canvas.paste(img, offset)
        return canvas

class PM25Dataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df = dataframe
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_name = os.path.join(IMG_DIR, self.df.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        label = int(self.df.iloc[idx, 1])
        if self.transform: image = self.transform(image)
        return image, label

# 3. MODEL INITIALIZATION

In [8]:
def get_model(name):
    if name == "mobilenet_v2":
        m = models.mobilenet_v2(weights='IMAGENET1K_V1')
        m.classifier[1] = nn.Linear(m.last_channel, 6)
    elif name == "resnet50":
        m = models.resnet50(weights='IMAGENET1K_V1')
        m.fc = nn.Linear(m.fc.in_features, 6)
    elif name == "efficientnet_b0":
        m = models.efficientnet_b0(weights='IMAGENET1K_V1')
        m.classifier[1] = nn.Linear(m.classifier[1].in_features, 6)
    elif name == "inception_v3":
        m = models.inception_v3(weights='IMAGENET1K_V1', aux_logits=True)
        m.fc = nn.Linear(m.fc.in_features, 6)
    elif name == "resnet101":
        m = models.resnet101(weights='IMAGENET1K_V1')
        m.fc = nn.Linear(m.fc.in_features, 6)
    return m.to(device)

# 4. EVALUATION FUNCTION

In [11]:
def evaluate_and_plot(model, loader, name, split_id):
    model.eval()
    y_true, y_pred, y_probs = [], [], []
    with torch.no_grad():
        for imgs, lbls in loader:
            imgs, lbls = imgs.to(device), lbls.to(device)
            out = model(imgs)
            y_true.extend(lbls.cpu().numpy())
            y_pred.extend(out.argmax(1).cpu().numpy())
            y_probs.extend(torch.softmax(out, 1).cpu().numpy())